# Synthetic Traces Plotting/Analysis

This notebook explores the input rate generation methods implemented in the `dfaas_input_rate` module. The generation is done in traces, one for each agent, at the beginning of the episode for all steps. There are two main types of traces:

* Traces generated from real data: **not covered** in this notebook.
* Traces generated from synthetic data: **covered** in this notebook.

Some constraints (or assumptions):

1. Each episode has a fixed length of 288 steps (each step represents 5 minutes in a 24-hour period).
2. For each step and agent, the input rate takes positive values in a fixed range (1, 150).

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import dfaas_input_rate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

## Sinusoidal traces

### Plot of a single trace

In [ ]:
rng = np.random.default_rng(seed=42)

In [ ]:
def plot_single_trace(rng):
    trace = dfaas_input_rate._gen_synthetic_sinusoidal(rng)

    plt.close(fig="single_trace_fixed_args")
    fig, axes = plt.subplots(2, 1, num="single_trace_fixed_args", layout="constrained", figsize=(8, 10))
    fig.canvas.header_visible = False

    ax = axes[0]  # First plot: arrival rate.
    ax.plot(trace)

    ax.set_title(f"Arrival rate")
    ax.set_ylabel("Function invocations")
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.set_ylim(bottom=0, top=150 + 5)
    ax.set_xlabel("Step")
    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

    ax.grid(axis="both")
    ax.set_axisbelow(True)

    ax = axes[1]  # Second plot: arrival rate distribution.
    bins = np.arange(1, 150 + 1)  # Since the range is [1, 150].
    ax.hist(trace, bins=bins, color="tab:blue", edgecolor="black")
    ax.set_title("Distribution of Invocations")
    ax.set_xlabel("Function invocations")
    ax.set_ylabel("Frequency")
    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.grid(axis="y")

    return fig


fig = plot_single_trace(rng)

### Compare multiple traces

**WARNING**: This section does not currently work!

In [ ]:
def make_traces(n_traces):
    assert n_traces > 0

    rng = np.random.default_rng(seed=42)

    # Fixed values for all traces.
    agents = ["node_0"]
    max_steps, min_reqs, max_reqs = 288, 0, 150
    limits = {"node_0": {"min": min_reqs, "max": max_reqs}}

    # Generate a specific seed for this trace.
    iinfo = np.iinfo(np.uint32)
    seed = rng.integers(0, high=iinfo.max, size=1)[0].item()
    specific_rng = np.random.default_rng(seed=seed)

    traces = []
    for i in range(n_traces):
        trace = dfaas_input_rate._gen_synthetic_sinusoidal(specific_rng)
        traces.append(trace)
    return traces


n_traces = 10
traces = make_traces(n_traces)

#### Dynamic Time Warping (DTW)

See: https://dynamictimewarping.github.io/python/

In [ ]:
from dtw import dtw

In [ ]:
def compute_dtw(traces):
    res_dtw, res_dtw_norm = [], []

    for x_index in range(len(traces)):
        x_dtw, x_dtw_norm = [], []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            dtw_xy = dtw(x, y)

            x_dtw.append(dtw_xy.distance)
            x_dtw_norm.append(dtw_xy.normalizedDistance)

        res_dtw.append(np.mean(x_dtw))
        res_dtw_norm.append(np.mean(x_dtw_norm))

    return np.mean(res_dtw), np.mean(res_dtw_norm)


dtw_distance, dtw_distance_norm = compute_dtw(traces)
dtw_distance_next, dtw_distance_next_norm = compute_dtw(traces_next)

print("Current traces:")
print("  Dynamic Time Warping distance (average on all traces) =", dtw_distance)
print("  Dynamic Time Warping normalized (average on all traces) =", dtw_distance_norm)
print()
print("Next traces:")
print("  Dynamic Time Warping distance (average on all traces) =", dtw_distance_next)
print("  Dynamic Time Warping normalized (average on all traces) =", dtw_distance_next_norm)

In [ ]:
def dtwPlotTwoWay(
    d,
    fig,
    xts=None,
    yts=None,
    offset=0,
    ts_type="l",
    match_indices=None,
    match_col="gray",
    xlab="Step",
    ylab="Invocations",
    **kwargs,
):
    """Customized version of dtwPlotTwoWay function for this notebook.

    Original code: https://github.com/DynamicTimeWarping/dtw-python/blob/master/dtw/dtwPlot.py
    """
    import matplotlib.pyplot as plt
    from matplotlib import collections as mc

    if xts is None or yts is None:
        try:
            xts = d.query
            yts = d.reference
        except:
            raise ValueError("Original timeseries are required")

    # ytso = yts + offset
    offset = -offset

    xtimes = np.arange(len(xts))
    ytimes = np.arange(len(yts))

    ax = fig.subplots()

    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)

    ax.plot(xtimes, np.array(xts), color="k", label="X", **kwargs)
    ax.plot(ytimes, np.array(yts) - offset, label="Y", **kwargs)  # Plot with offset applied
    fig.legend()

    if offset != 0:
        # Create an offset axis
        ax2 = ax.twinx()
        ax2.tick_params("y", colors="b")
        ql, qh = ax.get_ylim()
        ax2.set_ylim(ql + offset, qh + offset)

    # https://stackoverflow.com/questions/21352580/matplotlib-plotting-numerous-disconnected-line-segments-with-different-colors
    if match_indices is None:
        idx = np.linspace(0, len(d.index1) - 1)
    elif not hasattr(match_indices, "__len__"):
        idx = np.linspace(0, len(d.index1) - 1, num=match_indices)
    else:
        idx = match_indices
    idx = np.array(idx).astype(int)

    col = []
    for i in idx:
        col.append([(d.index1[i], xts[d.index1[i]]), (d.index2[i], -offset + yts[d.index2[i]])])

    lc = mc.LineCollection(col, linewidths=1, linestyles=":", colors=match_col)
    ax.add_collection(lc)

    return ax


def make_dtw_plot(traces):
    n = len(traces)

    x_index = ipywidgets.BoundedIntText(
        value=0,
        min=0,
        max=len(traces),
        description=f"X trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    y_index = ipywidgets.BoundedIntText(
        value=1,
        min=0,
        max=len(traces),
        description=f"Y trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    with plt.ioff():
        plt.close(fig="make_dtw_plot")
        fig = plt.figure(num="make_dtw_plot", layout="constrained")
    fig.canvas.header_visible = False

    out = ipywidgets.Output()

    def make_plot():
        x = traces[x_index.value]
        y = traces[y_index.value]

        dtw_xy = dtw(x, y, keep_internals=True)

        with out:
            out.clear_output()
            print(f"Distance: {dtw_xy.distance:.3f}")
            print(f"Distance normalized: {dtw_xy.normalizedDistance:.3f}")

        # Make the plot.
        fig.clear()

        dtwPlotTwoWay(dtw_xy, fig)

        fig.canvas.draw_idle()
        fig.canvas.draw_idle()

    # Make the initial plot with the default values.
    make_plot()

    # Link the input widgets to the plotting function.
    x_index.observe(lambda change: make_plot(), names="value")
    y_index.observe(lambda change: make_plot(), names="value")

    inputs = ipywidgets.VBox([x_index, y_index])

    return ipywidgets.AppLayout(
        header=ipywidgets.VBox([inputs, out]),
        center=fig.canvas,
        pane_heights=[0.9, 5, 0],
    )


make_dtw_plot(traces)

#### Mean Absolute Error (MAE)

In [ ]:
def compute_single_mae(x, y):
    assert len(x) == len(y)
    n = len(x)

    return (1 / n) * np.sum(np.abs(x - y))


def compute_mae(traces):
    mae = []

    for x_index in range(len(traces)):
        x_mae = []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            mae_xy = compute_single_mae(x, y)

            x_mae.append(mae_xy)

        mae.append(np.mean(x_mae))

    return np.mean(mae)


mae = compute_mae(traces)

print(f"Mean Absolute Error (average on all traces) = {mae:.3f}")

#### Mean Squared Error (MSE)

In [ ]:
def compute_single_mse(x, y):
    assert len(x) == len(y)
    n = len(x)

    return (1 / n) * np.sum(np.square(x - y))


def compute_mse(traces):
    mse = []

    for x_index in range(len(traces)):
        x_mse = []
        for y_index in range(len(traces)):
            if x_index == y_index:
                continue

            x = traces[x_index]
            y = traces[y_index]
            mse_xy = compute_single_mse(x, y)

            x_mse.append(mse_xy)

        mse.append(np.mean(x_mse))

    return np.mean(mse)


mse = compute_mse(traces)

print(f"Mean Square Error (average on all traces) = {mse:.3f}")

#### Root Mean Squared Error (RMSE)

In [ ]:
def compute_single_rmse(x, y):
    mse = compute_single_mse(x, y)
    return np.sqrt(mse)


def compute_rmse(traces):
    mse = compute_mse(traces)
    return np.sqrt(mse)


rmse = compute_rmse(traces)

print(f"Root Mean Square Error (average on all traces) = {rmse:.3f}")

#### Summary

In [ ]:
def make_basic_statistical_measures_app(traces):
    n = len(traces)

    x_index = ipywidgets.BoundedIntText(
        value=0,
        min=0,
        max=len(traces),
        description=f"X trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    y_index = ipywidgets.BoundedIntText(
        value=1,
        min=0,
        max=len(traces),
        description=f"Y trace [0, {n-1}]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    out = ipywidgets.Output()

    def make_widget():
        x = traces[x_index.value]
        y = traces[y_index.value]

        mae = compute_single_mae(x, y)
        mse = compute_single_mse(x, y)
        rmse = compute_single_rmse(x, y)

        with out:
            out.clear_output()
            print(f"Mean Absolute Error: {mae:.3f}")
            print(f"Mean Square Error: {mse:.3f}")
            print(f"Root Mean Square Error: {rmse:.3f}")

    # Make the initial plot with the default values.
    make_widget()

    # Link the input widgets to the plotting function.
    x_index.observe(lambda change: make_widget(), names="value")
    y_index.observe(lambda change: make_widget(), names="value")

    inputs = ipywidgets.VBox([x_index, y_index])

    return ipywidgets.AppLayout(header=inputs, center=out)


make_basic_statistical_measures_app(traces)

## Other traces

In [ ]:
def make_trace_plot(trace, plot_name="default"):
    for agent, trace in sorted(trace.items()):
        plt.close(fig=f"{plot_name}_trace_{agent}")
        fig = plt.figure(num=f"{plot_name}_trace_{agent}", layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        ax.plot(trace)

        ax.set_title(f"Function arrival rate (λ) ({agent = })")
        ax.set_ylabel("Rate")
        ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        ax.set_xlabel("Step")

        ax.grid(axis="both")
        ax.set_axisbelow(True)  # By default the axis is over the content.

### Constant trace

In [ ]:
def make_constant_trace():
    agents = ["node_0", "node_1"]
    max_steps = 288

    return dfaas_input_rate.synthetic_constant(max_steps, agents)


trace = make_constant_trace()

In [ ]:
make_trace_plot(trace, "constant")

### Linear growth trace

In [ ]:
def make_linear_growth_trace():
    agents = ["node_0", "node_1"]
    max_steps = 288

    return dfaas_input_rate.synthetic_linear_growth(max_steps, agents)


trace = make_linear_growth_trace()

In [ ]:
make_trace_plot(trace, "linear-growth")

### Step-change trace

In [ ]:
def make_step_change_trace():
    agents = ["node_0", "node_1"]
    max_steps = 288

    return dfaas_input_rate.synthetic_step_change(max_steps, agents)


trace = make_step_change_trace()

In [ ]:
make_trace_plot(trace, "step-change")

### Double linear growth trace

In [ ]:
def make_double_linear_growth_trace():
    agents = ["node_0", "node_1"]
    max_steps = 288

    return dfaas_input_rate.synthetic_double_linear_growth(max_steps, agents)


trace = make_double_linear_growth_trace()

In [ ]:
make_trace_plot(trace, "double-linear-growth")